In [134]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score,make_scorer,f1_score,classification_report
from sklearn.preprocessing import Normalizer,MinMaxScaler,StandardScaler,normalize
from sklearn.cross_validation import train_test_split

In [135]:
pd.set_option('max_columns',1000)

In [155]:
# df = pd.read_csv('../merged_caselevel_data.csv')
df = pd.get_dummies(pd.read_csv('final_feats_without_dummies.csv', low_memory=False))

In [156]:
df_y = df['dissentdummy'] #should be called 'label'. this is dummy

In [157]:
col_names = list(df.columns)

for col_name in col_names:
    if 'diss' in col_name:
        df.drop(col_name, axis=1, inplace=True)
        

df.drop('Unnamed: 0', axis=1, inplace=True)
# df.drop('dissentdummy', axis=1, inplace=True)
# df.drop('dissent', axis=1, inplace=True)

# to_drop_cols = ['unan', 'unanimous', 'dissentvalence', 'dissentvote', 'liberaldissent',
#            'dissmerdummy', 'dissmer', 'concur', 'concurdummy', 'diss0promerdummy']

# for to_drop in to_drop_cols:
#     df.drop(to_drop, axis=1, inplace=True)
    




In [158]:
#fill na's
df.fillna(0, inplace=True)

In [159]:
X=df.values
y=df_y.values

In [160]:
X_tr,X_te,y_tr,y_te=train_test_split(X,y,test_size=0.5,random_state=7)

In [161]:
print X_tr.shape
print y_tr.shape
print X_te.shape
print y_te.shape

(55833, 939)
(55833,)
(55833, 939)
(55833,)


In [162]:
##normalize train/test data
scaler = StandardScaler()
scaler.fit(X_tr)
# print X_te[:10]
# print x_mean.shape
X_te = scaler.transform(X_te)
# print X_te[:10]

In [163]:
paramgrid={'n_estimators':[10,50,100],'max_depth':[1,5,10,15]}

In [164]:
rf_clf=RandomForestClassifier(random_state=42)

In [165]:
scorer=make_scorer(accuracy_score)

In [166]:
%%time
gridclf=GridSearchCV(rf_clf,paramgrid,scoring=scorer,cv=3,verbose=1)

CPU times: user 35 µs, sys: 9 µs, total: 44 µs
Wall time: 39.1 µs


In [39]:
gridclf.fit(X_tr,y_tr)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] n_estimators=10, max_depth=1 ....................................
[CV] ........................... n_estimators=10, max_depth=1 -   0.6s
[CV] n_estimators=10, max_depth=1 ....................................
[CV] ........................... n_estimators=10, max_depth=1 -   0.6s
[CV] n_estimators=10, max_depth=1 ....................................
[CV] ........................... n_estimators=10, max_depth=1 -   0.6s
[CV] n_estimators=50, max_depth=1 ....................................
[CV] ........................... n_estimators=50, max_depth=1 -   1.6s
[CV] n_estimators=50, max_depth=1 ....................................
[CV] ........................... n_estimators=50, max_depth=1 -   1.6s
[CV] n_estimators=50, max_depth=1 ....................................
[CV] ........................... n_estimators=50, max_depth=1 -   1.6s
[CV] n_estimators=100, max_depth=1 ...................................
[CV] ...........

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  3.6min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 50, 100], 'max_depth': [1, 5, 10, 15]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=make_scorer(accuracy_score), verbose=2)

In [40]:
print gridclf.best_params_
print gridclf.best_score_

{'n_estimators': 100, 'max_depth': 15}
0.999749252234


In [27]:
gridclf.predict(X_te)

array([ 0.,  0.,  1., ...,  0.,  0.,  0.])

In [41]:
y_te_pred=gridclf.predict(X_te)

In [42]:
accuracy_score(y_te,y_te_pred)

0.99505668690559346

In [46]:
print classification_report(y_te, y_te_pred)

             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00     51399
        1.0       1.00      0.94      0.97      4434

avg / total       1.00      1.00      0.99     55833



In [167]:
rf_clf_f=RandomForestClassifier(random_state=42, n_estimators=100, max_depth=15)

rf_clf_f.fit(X_tr,y_tr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [168]:
importances = rf_clf_f.feature_importances_
col_names = list(df.columns)

def get_top_n(n, arr, prev_list=[]):
    
    if n <= 0:
        return
    
    most_imp = -1
    most_imp_index = -1

    for i in range(len(arr)):

        if i in prev_list:
            continue

        if arr[i] > most_imp:
            most_imp = arr[i]
            most_imp_index = i

    prev_list.append(most_imp_index)
            
    print col_names[most_imp_index]
    
    get_top_n(n - 1, arr, prev_list)
    
get_top_n(10, importances)
    

unan
unanimous
concurdummy
concur
lengthopin
Wopinionlenght
pagelgth
Wlengthopin
totalcites
Wtotalcites


In [169]:
y_te_pred = rf_clf_f.predict(X_te)

print classification_report(y_te, y_te_pred)

             precision    recall  f1-score   support

        0.0       0.96      1.00      0.98     51399
        1.0       0.96      0.51      0.66      4434

avg / total       0.96      0.96      0.95     55833



In [151]:
accuracy_score(y_te,y_te_pred)

0.92371894757580641